In [5]:
import pymongo
from datetime import timedelta, date
import charts

In [6]:
client = pymongo.MongoClient('localhost', 27017)
ganji = client['ganji']
item_info = ganji['item_info_new']

In [7]:
for i in item_info.find().limit(5):
    print(i)

{'title': 'LG 5KG滚筒洗衣机', 'cates': '北京二手家居百货', 'area': '北京-朝阳', 'price': '540', '_id': ObjectId('583fd80f6976954720b38b30'), 'url': 'http://zhuanzhuan.ganji.com/detail/790807830338469892z.shtml'}
{'title': 'Apple MacBookPro 15.4英寸 Retina', 'cates': '北京二手笔记本', 'area': '北京-朝阳', 'price': '1.2万', '_id': ObjectId('583fd80f6976954e4cf5aaf6'), 'url': 'http://zhuanzhuan.ganji.com/detail/794128261972918276z.shtml'}
{'title': '电饭锅一次没用', 'area': '北京-丰台', 'price': '35', 'cates': '北京二手家居百货', '_id': ObjectId('583fd80f6976954f784160f8'), 'url': 'http://zhuanzhuan.ganji.com/detail/789392669006544900z.shtml'}
{'_id': ObjectId('583fd80f697695213893c7f1'), 'cates': '北京二手老年用品', 'title': '不做微商了 货物便宜卖了', 'price': '168', 'area': '郴州-苏仙', 'url': 'http://zhuanzhuan.ganji.com/detail/798397243517059076z.shtml'}
{'title': '好太太自动晾衣架，全新未装未使用。家人买了就没有用，全新甩卖，限北京自提。', 'area': '北京-朝阳', 'price': '200', 'cates': '北京二手家具', '_id': ObjectId('583fd8106976954f784160f9'), 'url': 'http://zhuanzhuan.ganji.com/detail/77295384360129

In [8]:
for i in item_info.find().limit(5):
    print(i['area'])

北京-朝阳
北京-朝阳
北京-丰台
郴州-苏仙
北京-朝阳


In [9]:
area_list = []
for i in item_info.find():
    # print(i['area'].split('-')[0])
    if i['area'].split('-')[0] == '北京':
        # print(i['area'])
        area_list.append(i['area'])
area_index = list(set(area_list))
print(area_index)

['北京-北京周边', '北京-通州', '北京-崇文', '北京-昌平', '北京-丰台', '北京-海淀', '北京-房山', '北京-宣武', '北京-怀柔', '北京-东城', '北京-西城', '北京-密云', '北京-大兴', '北京-门头沟', '北京-朝阳', '北京-石景山', '北京-延庆', '北京-燕郊', '北京-平谷', '北京-顺义']


In [10]:
pipeline = [
    {'$match': {'area': '北京-朝阳'}},  
    # {'$match': {'$and': [{'cates': '北京二手家居百货'}, {'area': '北京-朝阳'}]}}, 
    {'$group': {'_id': '$cates', 'counts': {'$sum': 1}}},              # 数据重新进行分组 ，统计类目出现的次数
    {'$sort': {'counts': -1}},               # 1 是正序，-1 是倒序
]

In [11]:
line = 0
for i in item_info.aggregate(pipeline):
    line += 1
    print(i)
    if line == 3:
        break


{'counts': 456, '_id': '北京二手家居百货'}
{'counts': 454, '_id': '北京家用电器'}
{'counts': 446, '_id': '北京二手家具'}


In [12]:
'''
def data_gen(area):
    line = 0
    pipeline = [
        {'$match': {'area': area}},  
        {'$group': {'_id': '$cates', 'counts': {'$sum': 1}}},              
        {'$sort': {'counts': -1}},               
    ]
    for i in item_info.aggregate(pipeline):
        line += 1
        yield [i['_id'],i['counts']]
        if line == 3:
            break
'''

In [35]:
def data_gen(area,limit):
    pipeline = [
        {'$match': {'area': area}},  
        {'$group': {'_id': '$cates', 'counts': {'$sum': 1}}},
        {'$sort': {'counts': -1}},
        {'$limit': limit},                    # 前几行
    ]

    for i in item_info.aggregate(pipeline):
        data = {
            'name': i['_id'],
            'data': [i['counts']],
            'type': 'column'
        }
        yield data

In [42]:
for i in data_gen('北京-朝阳', 5):
    print(i)

{'type': 'column', 'data': [456], 'name': '北京二手家居百货'}
{'type': 'column', 'data': [454], 'name': '北京家用电器'}
{'type': 'column', 'data': [446], 'name': '北京二手家具'}
{'type': 'column', 'data': [340], 'name': '北京二手笔记本'}
{'type': 'column', 'data': [278], 'name': '北京二手图书/乐器/运动'}


In [41]:
area = '北京-朝阳'
limit = 5
series = [i for i in data_gen(area, limit)]

options = {
    'chart'   : {'zoomType':'xy'},
    'title'   : {'text': '发帖最多的 TOP5 类目'},
    'subtitle': {'text': area},
     'yAxis'   : {'title': {'text': '数量'}}
    }

charts.plot(series,options=options,show='inline')